In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Database connection and query
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

df_clients = pd.read_sql_query("SELECT * FROM core_client", conn)
df_sales = pd.read_sql("SELECT * FROM core_sale;", conn)
df_salesmans = pd.read_sql("SELECT * FROM core_salesman;", conn)
df_categories = pd.read_sql("SELECT * FROM core_category;", conn)
df_products = pd.read_sql("SELECT * FROM core_product;", conn)
df_productsales = pd.read_sql("SELECT * FROM core_productsale;", conn)
df_productsales

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn

,id,quantity,income,product_id,sale_id,income_currency
0,16921,4.10,14.35,17,00034279,USD
1,16953,4.00,14.00,17,00034290,USD
2,16930,8.10,43.52,22,00034282,USD
3,17005,6.80,21.87,23,00034311,USD
4,16906,24.00,46.72,33,00034269,USD
...,...,...,...,...,...,...
71828,18772,25.30,0.00,489,00034825,USD
71829,19432,4.30,0.00,489,00035023,USD
71830,19630,8.70,0.00,489,00035079,USD
71831,59878,4.40,0.00,489,00001021,USD


In [3]:
# Quantity of sales a salesman made with every client

df_merged = pd.merge(df_sales, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_sale', '_salesman'))
df_merged_clients = pd.merge(df_merged, df_clients, left_on='client_id', right_on='id', suffixes=('_sale', '_client'))
df_merged_clients.drop(columns=['id_sale','id_salesman','date', 'status', 'income_currency', 'description', 'identity_card_sale', 'image_sale', 'phone_1', 'phone_2', 'address_sale', 'identity_card_client', 'phone', 'address_client', 'image_client'], inplace=True)
df_merged_clients.rename(columns={'name_sale': 'name_salesman'}, inplace=True)
df_merged_clients.set_index(['salesman_id', 'client_id', 'name_client'], inplace=True)
df_merged_clients["count"] = df_merged_clients.groupby(['salesman_id' ,'client_id'])['id'].count()
df_merged_clients["total_income"] = df_merged_clients.groupby(['salesman_id' ,'client_id'])['income'].sum()
df_merged_clients.reset_index(inplace=True)
data = {
    "name_salesman": df_merged_clients.groupby(['salesman_id' ,'client_id'])['name_salesman'].first(),
    "name_client": df_merged_clients.groupby(['salesman_id' ,'client_id'])['name_client'].first(),
    "count": df_merged_clients.groupby(['salesman_id' ,'client_id'])['count'].first(),
    'total_income': df_merged_clients.groupby(['salesman_id' ,'client_id'])['total_income'].first()
}
df_aux = pd.DataFrame(data)
# df_aux.reset_index(inplace=True)
df_aux

name_salesman  \
salesman_id client_id                   
1           2          VENTAS DIRECTA   
            3          VENTAS DIRECTA   
            5          VENTAS DIRECTA   
            8          VENTAS DIRECTA   
            9          VENTAS DIRECTA   
...                               ...   
14          1034         Gunner Oviol   
            1061         Gunner Oviol   
15          32               inactivo   
            320              inactivo   
            961              inactivo   

                                                             name_client  \
salesman_id client_id                                                      
1           2                                           VECINO DEFREITAS   
            3                                                     KALIFA   
            5                                            VENTAS DIRECTAS   
            8                                VICTOR JOSE NAVEDA PARTIDAS   
            9                              RAMIRO JOSE VILLALOBOS BLANCO   
...                                                                  ...   
14          1034       CELIA VICTORIA COLINA BRAVO (DANNAÑS MARKET DE...   
            1061                     MARIA AUXILIADORA JELABI DE BEUVIRN   
15          32                     DAVIDE DA CONCEICAO NUNES (AVEIRENSE)   
            320        PANADERIA Y PASTELERIA EL RINCON DEL TACHIRA J...   
            961        FRANK ROBERT ROMERO SECO (ABASTO Y LICORERIA M...   

                       count  total_income  
salesman_id client_id                       
1           2             16       6900.48  
            3              1         10.90  
            5             36     136310.72  
            8              1         11.44  
            9              8         95.01  
...                      ...           ...  
14          1034           5        169.54  
            1061           2        148.34  
15          32             1         40.51  
            320            3         20.76  
            961            1         42.19  

[1499 rows x 4 columns]

In [4]:
# Quantity and income for categories for a salesman

df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_merged_with_categories = pd.merge(df_merged_with_products, df_categories, left_on='category_id', right_on='id', suffixes=('_products', '_categories'))
df_final_merged = pd.merge(df_merged_with_categories, df_clients, left_on='client_id', right_on='id', suffixes=('_categories', '_client'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card', 'image_client',
                    'address', 'phone', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'image_categories', 'id', 'id_categories',
                    'income_sale', 'id_sale', 'id_products',
                    'client_id'
                    ], inplace=True)
df_final_merged.set_index(['salesman_id', 'category_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['salesman_id', 'category_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['salesman_id', 'category_id'])['income_productsales'].sum()
df_final_merged.reset_index(inplace=True)
data = {
    "name_salesman": df_final_merged.groupby(['salesman_id', 'category_id'])['name'].first(),
    "name_category": df_final_merged.groupby(['salesman_id', 'category_id'])['name_categories'].first(),
    "salesman_id": df_final_merged.groupby(['salesman_id', 'category_id'])['salesman_id'].first(),
    "category_id": df_final_merged.groupby(['salesman_id', 'category_id'])['category_id'].first(),
    "count": df_final_merged.groupby(['salesman_id', 'category_id'])['count'].first(),
    "total_income": df_final_merged.groupby(['salesman_id', 'category_id'])['total_income'].first()
}

df_aux_2 = pd.DataFrame(data)
# df_aux_2.drop(columns=['category_id', 'client_id'], inplace=True)
# df_aux_2.reset_index(inplace=True)
df_aux_2

name_salesman  \
salesman_id category_id                                                      
1           8                                        HIPERMERCADO LHAU C.A   
            16                   MORICE CHACHATI (INVERSIONES LATINA 2011)   
            17             COMERCIALIZADORA CHARCUTERIA Y BODEGON SKY, C.A   
            19                                       HIPERMERCADO LHAU C.A   
            20                                       HIPERMERCADO LHAU C.A   
...                                                                    ...   
14          1064            PANADERIA Y PASTELERIA LA GRAN COSTA NOVA, C.A   
15          19           PANADERIA Y PASTELERIA EL RINCON DEL TACHIRA J...   
            23                       DAVIDE DA CONCEICAO NUNES (AVEIRENSE)   
            45           FRANK ROBERT ROMERO SECO (ABASTO Y LICORERIA M...   
            47           PANADERIA Y PASTELERIA EL RINCON DEL TACHIRA J...   

                                  name_category  salesman_id  category_id  \
salesman_id category_id                                                     
1           8                 CREMAS Y NATILLAS            1            8   
            16                         CHORIZOS            1           16   
            17                        JAMONERIA            1           17   
            19              BOLOGNAS Y FIAMBRES            1           19   
            20                   PAVOS Y POLLOS            1           20   
...                                         ...          ...          ...   
14          1064                     CONFITERIA           14         1064   
15          19              BOLOGNAS Y FIAMBRES           15           19   
            23                   QUESOS FRESCOS           15           23   
            45           CHOCOLATES Y GOLOSINAS           15           45   
            47                         GALLETAS           15           47   

                            count  total_income  
salesman_id category_id                          
1           8             7313.00      15264.29  
            16             130.25        858.13  
            17            3434.17      20729.16  
            19           12650.74      52818.89  
            20             235.81       1772.89  
...                           ...           ...  
14          1064            44.00        222.68  
15          19               3.67         15.39  
            23               6.42         40.51  
            45               9.00         42.19  
            47               6.00          5.40  

[456 rows x 6 columns]

In [6]:
# Quantity and income for products for a salesman

df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_final_merged = pd.merge(df_merged_with_products, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_sales_products', '_salesmans'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card',
                    'address', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'income_sale', 'id_sale', 'image_sales_products',
                    'category_id', 'image_salesmans', 'phone_1', 'phone_2', 'client_id'
                    ], inplace=True)
df_final_merged.set_index(['salesman_id', 'product_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['salesman_id', 'product_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['salesman_id', 'product_id'])['income_productsales'].sum()

# df_final_merged.reset_index(inplace=True)
# df_final_merged[df_final_merged['salesman_id'] == 15]

df_final_merged

id_productsales  quantity  income_productsales  \
salesman_id product_id                                                   
4           489                   16961     24.00                 0.00   
            489                   16924      4.30                 0.00   
            489                   16929     28.00                 0.00   
            489                   16932     24.00                 0.00   
            489                   16933     30.00                 0.00   
...                                 ...       ...                  ...   
15          141                   87363      1.00                 0.84   
            143                   74490      1.00                 0.96   
            143                   74493      1.00                 0.96   
            143                   87362      1.00                 0.96   
            355                   74492      3.67                15.39   

                         sale_id  id_sales_products  \
salesman_id product_id                                
4           489         00034292                489   
            489         00034279                489   
            489         00034282                489   
            489         00034283                489   
            489         00034283                489   
...                          ...                ...   
15          141           000709                141   
            143         NE007580                143   
            143         NE007582                143   
            143           000709                143   
            355         NE007580                355   

                                   name_sales_products  id_salesmans  \
salesman_id product_id                                                 
4           489                                   NONE             4   
            489                                   NONE             4   
            489                                   NONE             4   
            489                                   NONE             4   
            489                                   NONE             4   
...                                                ...           ...   
15          141         NACHOS ORIGINAL 150GR MARIACHI            15   
            143          NACHOS CHEDDAR 150GR MARIACHI            15   
            143          NACHOS CHEDDAR 150GR MARIACHI            15   
            143          NACHOS CHEDDAR 150GR MARIACHI            15   
            355           FIAMBRE DE CERDO FRIO CARNES            15   

                       name_salesmans   count  total_income  
salesman_id product_id                                       
4           489          Luis Guanipa  978.00          0.00  
            489          Luis Guanipa  978.00          0.00  
            489          Luis Guanipa  978.00          0.00  
            489          Luis Guanipa  978.00          0.00  
            489          Luis Guanipa  978.00          0.00  
...                               ...     ...           ...  
15          141              inactivo    3.00          2.52  
            143              inactivo    3.00          2.88  
            143              inactivo    3.00          2.88  
            143              inactivo    3.00          2.88  
            355              inactivo    3.67         15.39  

[71833 rows x 10 columns]